In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [2]:
import os
import re
doc_list = []
for filename in os.listdir('./kanerai_docs_for_columbia_confidential'):
    if filename.endswith('.pdf'):
        doc_list.append('./kanerai_docs_for_columbia_confidential/' + filename)

In [3]:
doc_list

['./kanerai_docs_for_columbia_confidential/TEL133.20170810.dce07916-3ca5-2704-db92-fcfd7bbd6f53.5d35d0d4f9a56c3750c8880591dcb580.pdf',
 './kanerai_docs_for_columbia_confidential/MADPK24.20161220.d24b2b6d-896b-5b3b-6c6c-8e20884d8f2d.058b1a3ed738f0e375bf16c0295491f8.pdf',
 './kanerai_docs_for_columbia_confidential/DRYDN50.20170829.5bd363e3-734b-6a03-63cf-a3d63a1fb347.d1c70f9461000315525cf47580109a0f.pdf',
 './kanerai_docs_for_columbia_confidential/CIFLO.20150315.5146fb18-0734-b611-3cf0-7d1572ebb912..pdf',
 './kanerai_docs_for_columbia_confidential/DRYDN64.20180501.c9d74b60-e118-c916-1310-e6ccbd5b0388.3678beae0a66886984c4d63f593ca2d6.pdf',
 './kanerai_docs_for_columbia_confidential/HALCL132.20130711.486eb8ec-8cd5-f18f-2811-7aad41f02ac7.d72d91a2922b641dec9f58e996156295.pdf',
 './kanerai_docs_for_columbia_confidential/WEBPK.20180720.6f287ea2-3507-fec4-973c-6add2abe0486.f07df36bc6ebf22c6ba11c9cbd95507b.pdf',
 './kanerai_docs_for_columbia_confidential/MARIL174.20171026.7721cdcd-645d-8138-bce5

In [15]:
import time
text={}
success=0
fail=0

start = time.time()
for file_name in doc_list[:5]:
    try:
        text[file_name] = convert_pdf_to_txt(file_name)
        success+=1
    except:
        fail+=1
        pass

end = time.time()
end - start

364.1512622833252

In [16]:
success,fail

(4, 1)

In [17]:
6/4*260

390.0

In [ ]:
text={}
success=0
fail=0
for file_name in doc_list:
    try:
        text[file_name] = convert_pdf_to_txt(file_name)
        success+=1
    except:
        fail+=1
        print(file_name)
        pass

./kanerai_docs_for_columbia_confidential/ALM7R2.20181015.8fa5fe2d-f705-21fa-9693-ccfdd2360394.578059488c4f8264b3224bb80c88d844.pdf
./kanerai_docs_for_columbia_confidential/ALM17.20180716.91efb517-bc1f-7469-c548-bd8a23ef79a4.b4ea9f7e2ec4c799e6744c66197f5e19.pdf
./kanerai_docs_for_columbia_confidential/JMPCA3R.20190125.e0eb03ab-00c7-da66-57b4-b9250f60f899.0004e7c3f5e92167b59ae071ae839ea7.pdf
./kanerai_docs_for_columbia_confidential/RRCL1.20170804.8d8d399e-90d0-3544-8918-90a940bcafe7.5049a5f2f4e49e1f98579b63168dc7ad.pdf
./kanerai_docs_for_columbia_confidential/CATLK13.20171017.e063fc0c-8b4b-0c97-0a08-46019a74198f.b061de595c6a1a460e14e0245e900bbd.pdf
./kanerai_docs_for_columbia_confidential/ACASL132.20170725.f6e81d3b-671f-e7b7-63ef-4f6258f64bf8.be9827c3def852e86013998c0941d98f.pdf
./kanerai_docs_for_columbia_confidential/GARR151.20170921.2c9652a8-01da-75aa-c0b3-2d9be16f357a.a8ead2d19d95f40017f562717a2c8829.pdf
./kanerai_docs_for_columbia_confidential/VOY1501.20171221.1b0eab6d-86e7-6a5c-f5c

In [51]:
print(success,fail)

260 52


In [18]:
len(text)

260

In [19]:
def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [20]:
defn={}
for doc in text:
    try:
        l=re.findall(r'(\“.+?\”)\:',text[doc])
        definition={}
        for i in range(len(l)-1):
            first=l[i]
            last=l[i+1]
            d=find_between_r(text[doc], first+':', last+':' )
            definition[first]=d
        defn[doc]=definition
    except:
        pass

In [21]:
for i in defn:
    if defn[i]=={}:
        try:
            defn_part=find_between_r( text[i], '\n\nDEFINITIONS \n\n', '\n\n1.2 \n\n' )
            l=re.findall(r'\n\n(\".+?\") ',defn_part)
            definition={}
            for j in range(len(l)-1):
                first=l[j]
                last=l[j+1]
                d=find_between_r(defn_part, '\n\n'+first, '\n\n'+last )
                definition[first]=d
            defn[i]=definition
        except:
            pass

In [22]:
for i in defn:
    if defn[i]=={}:
        print(i)

./kanerai_docs_for_columbia_confidential/GSCCDO7.20060511.bc76ecce-9bc5-3195-beaf-211905864537.702a9f5d6ab9f649c89e06eab622d3d3.pdf
./kanerai_docs_for_columbia_confidential/ARROP142.20171020.e2f54cae-545d-0923-7ff5-956b10192820.52ca4243b4c26dbdfe7a9498e739ab9d.pdf
./kanerai_docs_for_columbia_confidential/LATCL2.20060803.1555c425-1cac-b5e6-504d-d75206f606c2.9512e892a2d911ba8848bfbd7ba6ac18.pdf
./kanerai_docs_for_columbia_confidential/NZCGF_1.20180228.6dc1c422-d0a6-61db-d654-b86df47eb027.4163b9ca15ac10392e4922fd9f6d44cc.pdf
./kanerai_docs_for_columbia_confidential/SFORK.20050315.b2670dee-f3ec-04f7-1fda-b0add65f8c81..pdf
./kanerai_docs_for_columbia_confidential/FORTCO11.20180416.0ac9ed72-56f5-c5cd-c5ff-7dfba65b1c34.24647f121e25ca09ef51b61c7faca023.pdf
./kanerai_docs_for_columbia_confidential/MARIL174.20171026.7721cdcd-645d-8138-bce5-b8715f20c29c.55335e34a509a81c86b72554a4f35975.pdf
./kanerai_docs_for_columbia_confidential/NEUB15.20171017.174f779a-243a-2b20-7ad1-a88f817ee0d8.8c9c7ec497e16b

In [23]:
for i in defn:
    if defn[i]=={}:
        try:
            defn_part=find_between_r( text[i], '\n\nSection 1.01  Definitions.', '\n\nSection 1.02' )
            l=re.findall(r'\n\n(\“.+?\”) ',defn_part)
            definition={}
            for j in range(len(l)-1):
                first=l[j]
                last=l[j+1]
                d=find_between_r(defn_part, first, last )
                definition[first]=d
            if definition!={}:
                print(i)
            defn[i]=definition
        except:
            pass

./kanerai_docs_for_columbia_confidential/CIFLO.20150315.5146fb18-0734-b611-3cf0-7d1572ebb912..pdf


In [38]:
defn['./kanerai_docs_for_columbia_confidential/AVERYPT6.20190118.70b8478a-cb4f-5f98-e686-bbb9e8146d28.65b1399a07d1ede3c178013d4980b0c3.pdf']


{'“17g-5 Information”': '  The meaning specified in Section 14.16. \n\n',
 '“17g-5 Website”': '  A password-protected internet website which shall initially be \nlocated  at  https://www.structuredfn.com.    Any  change  of  the  17g-5  Website  shall  only  occur \nafter notice has been delivered by the Issuer to the Information Agent, the Trustee, the Collateral \nAdministrator, the Portfolio Manager, the Initial Purchaser, and the Rating Agencies setting the \ndate of change and new location of the 17g-5 Website. \n\n',
 '“Accountants’ Report”': '  An agreed-upon procedures report of the firm or firms \n\nappointed by the Issuer pursuant to Section 10.9(a). \n\n',
 '“Accounts”': '    Each  of  (i) the  Payment  Account,  (ii) the  Collection  Account, \n(iii) the  Ramp-Up  Account,  (iv)  the  Subordinated  Note  Ramp-Up  Account,  (v)  the  Revolver \nFunding  Account,  (vi) the  Expense  Reserve  Account,  (vii) the  Reserve  Account,  (viii) the \nCustodial Account, (ix) the Ongo

In [25]:
s = convert_pdf_to_txt('./kanerai_docs_for_columbia_confidential/MADPK17.20170623.2265775d-f6dd-f026-0694-eec88bac1514.8dca9892ca5c8b2778272c52780baae7.pdf')

In [26]:
s=s.replace('\n', '')
s=s.replace('\xa0', ' ')

In [27]:
directory=re.findall(r'(Section [0-9]+\.[0-9]+\.  )([^\.]+)\.+[\.| ]+[0-9]+',s)
directory

[('Section 1.1.  ', 'Definitions'),
 ('Section 1.2.  ', 'Assumptions as to Pledged Obligations'),
 ('Section 2.1.  ', 'Forms Generally'),
 ('Section 2.2.  ', 'Forms of Notes'),
 ('Section 2.3.  ', 'Authorized Amount; Stated Maturity; Denominations'),
 ('Section 2.4.  ', 'Additional Notes'),
 ('Section 2.5.  ', 'Execution, Authentication, Delivery and Dating'),
 ('Section 2.6.  ', 'Registration, Registration of Transfer and Exchange'),
 ('Section 2.7.  ', 'Mutilated, Defaced, Destroyed, Lost or Stolen Note'),
 ('Section 2.8.  ',
  'Payment of Principal and Interest and Other Amounts; Principal and Interest Rights Preserved'),
 ('Section 2.9.  ', 'Persons Deemed Owners'),
 ('Section 2.10.  ', 'Purchase and Surrender of Notes; Cancellation'),
 ('Section 2.11.  ', 'Depository Not Available'),
 ('Section 2.12.  ',
  'Notes Beneficially Owned by Non-Permitted Holders or in Violation of ERISA Representations'),
 ('Section 2.13.  ',
  'Deduction or Withholding from Payments on Notes; No Gross-

In [28]:
part={}
for i in range(len(directory)-1):
    first=directory[i][0]
    name=directory[i][1]
    last=directory[i+1][0]
    part[name]=find_between_r(text, first, last)

In [39]:
part

{'Definitions': 'Definitions.  Except as otherwise specified herein or as the context may otherwise require, terms defined in Annex A hereto shall have the respective meanings set forth in Annex A for all purposes of this Indenture, and the definitions of such terms are equally 758924-4-479-v17.0  - 2 - 80-40587115 \x0c applicable both to the singular and plural forms of such terms and to the masculine, feminine and neuter genders of such terms.  The word "including" shall mean "including without limitation."  All  references  in  this  Indenture  to  designated  "Articles,"  "Sections,"  "Subsections"  and  other subdivisions are to the designated articles, sections, subsections and other subdivisions of this Indenture.  The words "herein," "hereof," "hereunder" and other words of similar import refer to  this  Indenture  as  a  whole  and  not  to  any  particular  article,  section,  subsection  or  other subdivision. ',
 'Assumptions as to Pledged Obligations': 'Assumptions  as  to

In [50]:
sample = convert_pdf_to_txt('./kanerai_docs_for_columbia_confidential/AVERYPT6.20190118.70b8478a-cb4f-5f98-e686-bbb9e8146d28.65b1399a07d1ede3c178013d4980b0c3.pdf')
sample=sample.replace('\n', '')
sample=sample.replace('\xa0', ' ')
directory=re.findall(r'(Section [0-9]+\.[0-9]+\.  )([^\.]+)\.+[\.| ]+[0-9]+',sample)
directory

[('Section 2.11.  ', 'Definitive Notes'),
 ('Section 2.12.  ',
  'Notes Beneficially Owned by Persons Not QIB/QPs or IAI/QPs or in Violation of ERISA Representations'),
 ('Section 2.13.  ',
  'Deduction or Withholding from Payments on Notes; No Gross Up'),
 ('Section 2.14.  ', 'Delayed Draw Notes '),
 ('Section 4.2.  ',
  'Americas 90516197 (2K)  Satisfaction and Discharge of Indenture'),
 ('Section 5.10.  ', 'Restoration of Rights and Remedies'),
 ('Section 5.11.  ', 'Rights and Remedies Cumulative'),
 ('Section 5.12.  ', 'Delay or Omission Not Waiver'),
 ('Section 5.13.  ', 'Control by Majority of Controlling Class'),
 ('Section 5.14.  ', 'Waiver of Past Defaults'),
 ('Section 5.15.  ', 'Undertaking for Costs'),
 ('Section 5.16.  ', 'Waiver of Stay or Extension Laws'),
 ('Section 5.18.  ', 'Action on the Notes'),
 ('Section 6.10.  ', 'Acceptance of Appointment by Successor Trustee'),
 ('Section 6.11.  ',
  'Merger, Conversion, Consolidation or Succession to Business of Trustee'),
 ('

In [54]:
all_dir={}
for doc in text:
    try:
        t=text[doc]
        t=t.replace('\n', '')
        t=t.replace('\xa0', ' ')
        d=re.findall(r'(Section [0-9]+\.[0-9]+\.  )([^\.]+)\.+[\.| ]+[0-9]+',t)
        all_dir[doc]=d
    except:
        pass

In [55]:
all_dir

{'./kanerai_docs_for_columbia_confidential/DENALI11.20181129.b57e37d3-acee-4611-d8c8-7c76ba8a9386.47355130d719560161bbc56109c40b2c.pdf': [('Section 2.6.  ',
   'Mutilated, Destroyed, Lost, or Stolen Note'),
  ('Section 2.10.  ', 'Certificated Notes'),
  ('Section 2.13.  ', 'Additional Issuance'),
  ('Section 5.10.  ', 'Restoration of Rights and Remedies'),
  ('Section 5.11.  ', 'Rights and Remedies Cumulative'),
  ('Section 5.12.  ', 'Delay or Omission Not Waiver'),
  ('Section 5.13.  ', 'Control by Majority of the Controlling Class'),
  ('Section 5.14.  ', 'Waiver of Past Defaults'),
  ('Section 5.15.  ', 'Undertaking for Costs'),
  ('Section 5.16.  ', 'Waiver of Stay or Extension Laws'),
  ('Section 5.17.  ', 'Sale of Collateral'),
  ('Section 5.18.  ', 'Action on the Notes'),
  ('Section 6.5.  ', 'May Hold Notes'),
  ('Section 6.6.  ', 'Money Held in Trust'),
  ('Section 6.10.  ', 'Acceptance of Appointment by Successor'),
  ('Section 6.11.  ',
   'Merger, Conversion, Consolidation,

In [56]:
len(all_dir)

260

In [71]:
str(defn['./kanerai_docs_for_columbia_confidential/AVERYPT6.20190118.70b8478a-cb4f-5f98-e686-bbb9e8146d28.65b1399a07d1ede3c178013d4980b0c3.pdf'].keys())

"dict_keys(['“17g-5 Information”', '“17g-5 Website”', '“Accountants’ Report”', '“Accounts”', '“Accredited  Investor”', '“Act” and “Act of Holders”', '“Additional Notes”', '“Additional  Notes  Closing  Date”', '“Adjusted Collateral Principal Amount”', '“Administration Agreement”', '“Administrative Expense Cap”', '“Administrative Expenses”', '“Administrator”', '“Advance”', '“Affiliate” or “Affiliated”', '“Agent  Members”', '“Aggregate Outstanding Amount”', '“Aggregate Principal Balance”', '“Aggregate Ramp-Up Par Amount”', '“Aggregate Ramp-Up Par Condition”', '“Aggregate  Risk-Adjusted  Par  Amount”', '“Aggregated Reinvestment”', '“Applicable Issuer” or “Applicable Issuers”', '“Asset Quality Matrix”', '“Asset Quality Matrix Combination”', '“Assets”', '“Assumed Reinvestment Rate”', '“Authenticating Agent”', '“Authorized Denominations”', '“Authorized Officer”', '“Average  Life”', '“Balance”', '“Bank”', '“Bankruptcy Exchange”', '“Bankruptcy Exchange Test”', '“Bankruptcy Law”', '“Bankruptcy S

In [100]:
import sqlite3
sqlite_file = 'database.sqlite'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

c.execute('''CREATE TABLE Table1 (
            "Term" TEXT, 
            "Explanation" TEXT
            )''')

OperationalError: table Table1 already exists

In [60]:
for term,explanation in defn['./kanerai_docs_for_columbia_confidential/AVERYPT6.20190118.70b8478a-cb4f-5f98-e686-bbb9e8146d28.65b1399a07d1ede3c178013d4980b0c3.pdf'].items():
    c.execute("INSERT INTO Table1 VALUES (?,?)", (term, explanation))


In [61]:
c.execute("SELECT * FROM Table1")

In [62]:
c.fetchall()

[('“17g-5 Information”', '  The meaning specified in Section 14.16. \n\n'),
 ('“17g-5 Website”',
  '  A password-protected internet website which shall initially be \nlocated  at  https://www.structuredfn.com.    Any  change  of  the  17g-5  Website  shall  only  occur \nafter notice has been delivered by the Issuer to the Information Agent, the Trustee, the Collateral \nAdministrator, the Portfolio Manager, the Initial Purchaser, and the Rating Agencies setting the \ndate of change and new location of the 17g-5 Website. \n\n'),
 ('“Accountants’ Report”',
  '  An agreed-upon procedures report of the firm or firms \n\nappointed by the Issuer pursuant to Section 10.9(a). \n\n'),
 ('“Accounts”',
  '    Each  of  (i) the  Payment  Account,  (ii) the  Collection  Account, \n(iii) the  Ramp-Up  Account,  (iv)  the  Subordinated  Note  Ramp-Up  Account,  (v)  the  Revolver \nFunding  Account,  (vi) the  Expense  Reserve  Account,  (vii) the  Reserve  Account,  (viii) the \nCustodial Account, 

In [121]:
import sqlite3
sqlite_file = 'db1.sqlite'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.execute("CREATE TABLE DEFINATION1('Name' TEXT, 'Term' TEXT, 'Explanation' TEXT)")

In [ ]:
for file in defn:
    a=defn[file]
    for file_term in a.keys():
        file_term_defination=a[file_term]
        b=str(file)
        d=str(a)
        e=str(file_term_defination)
        c.execute("INSERT INTO DEFINATION1 VALUES (?,?,?)", (b,d,e))